# Week 4 Lab: Building the AMU Chatbot Brain

Your mission today, should you choose to accept it, is to build the "brain" for a new AMU chatbot. 

The goal is **Intent Classification**: We need to take a student's query (e.g., *"Où est mon emploi du temps?"*) and map it to a specific service from the AMU portal (e.g., `get_schedule`).

Today, we will build and compare **four** different "brains" to see how they perform. We'll compare them on two key metrics:
1.  **Accuracy**: Does it get the right answer?
2.  **Latency**: How fast is it?

## Module 0: Setup

First, let's install the libraries we'll need. We'll use `transformers`, `datasets` (for later), `scikit-learn` for our classic ML model, `pandas` for our final analysis, and `sentence-transformers` (which is built on top of `transformers`) for high-quality embeddings.

In [1]:
#pip install -U transformers datasets scikit-learn pandas sentence-transformers accelerate torch

In [2]:
import time
import numpy as np
import pandas as pd
import re
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
from sentence_transformers import SentenceTransformer
import torch

# A helper to print things nicely
from IPython.display import display, Markdown

## Module 1: The Dataset - Our "Ground Truth"

Before we can build a classifier, we need data! Based on the AMU portal, we'll define 5 key intents. We'll then create a small, synthetic dataset of *training* queries and *testing* queries.

**Our 5 Intents:**
1.  `get_schedule` (for "Planning des cours (ADE)")
2.  `check_email` (for "Ma messagerie")
3.  `register_classes` (for "Inscriptions pédagogiques (IP)")
4.  `get_student_card` (for "Ma carte AMU")
5.  `find_library_info` (for "BU" / "Compte Lecteur BU")

In [3]:
# Here is our training data. We'll use this to train Classifier 2.
train_data = [
    {"text": "Où est mon emploi du temps?", "label": "get_schedule"},
    {"text": "Je veux voir mes cours de demain", "label": "get_schedule"},
    {"text": "Afficher mon planning de la semaine", "label": "get_schedule"},
    {"text": "J'ai un nouveau mail?", "label": "check_email"},
    {"text": "Ouvrir ma messagerie", "label": "check_email"},
    {"text": "Boite de réception", "label": "check_email"},
    {"text": "Comment je m'inscris à un cours?", "label": "register_classes"},
    {"text": "Où sont les inscriptions pédas?", "label": "register_classes"},
    {"text": "Je dois faire mon IP", "label": "register_classes"},
    {"text": "J'ai perdu ma carte étudiante", "label": "get_student_card"},
    {"text": "Refaire ma carte AMU", "label": "get_student_card"},
    {"text": "La BU est ouverte?", "label": "find_library_info"},
    {"text": "Quels sont les horaires de la bibliothèque?", "label": "find_library_info"},
    {"text": "Je veux emprunter un livre", "label": "find_library_info"}
]

# Here is our test data. We'll use this to evaluate ALL classifiers.
test_data = [
    {"text": "C'est quand mon prochain TD?", "label": "get_schedule"},
    {"text": "Ouvrir la boite de réception", "label": "check_email"},
    {"text": "Je veux m'inscrire en L3", "label": "register_classes"},
    {"text": "Ma carte est cassée", "label": "get_student_card"},
    {"text": "Les horaires de la BU St Charles", "label": "find_library_info"},
    {"text": "Quelle salle pour mon cours de 10h?", "label": "get_schedule"},
    {"text": "J'ai reçu un email important?", "label": "check_email"},
    {"text": "C'est quand les IP?", "label": "register_classes"},
    {"text": "Où est-ce que je peux imprimer avec ma carte?", "label": "get_student_card"},
    {"text": "Comment réserver un livre à la BU?", "label": "find_library_info"}
]

# Let's put them in a DataFrame to visualize
train_df = pd.DataFrame(train_data)
test_df = pd.DataFrame(test_data)

display(Markdown("### Training Data"))
display(train_df)
display(Markdown("### Test Data"))
display(test_df)

### Training Data

,text,label
0,Où est mon emploi du temps?,get_schedule
1,Je veux voir mes cours de demain,get_schedule
2,Afficher mon planning de la semaine,get_schedule
3,J'ai un nouveau mail?,check_email
4,Ouvrir ma messagerie,check_email
5,Boite de réception,check_email
6,Comment je m'inscris à un cours?,register_classes
7,Où sont les inscriptions pédas?,register_classes
8,Je dois faire mon IP,register_classes
9,J'ai perdu ma carte étudiante,get_student_card


### Test Data

,text,label
0,C'est quand mon prochain TD?,get_schedule
1,Ouvrir la boite de réception,check_email
2,Je veux m'inscrire en L3,register_classes
3,Ma carte est cassée,get_student_card
4,Les horaires de la BU St Charles,find_library_info
5,Quelle salle pour mon cours de 10h?,get_schedule
6,J'ai reçu un email important?,check_email
7,C'est quand les IP?,register_classes
8,Où est-ce que je peux imprimer avec ma carte?,get_student_card
9,Comment réserver un livre à la BU?,find_library_info


--- 
## Module 2: Classifier 1 - The Regex Baseline

Our first model isn't a model at all! It's a simple function using Regular Expressions (or just `if/in` statements) to check for keywords.

**Why?** It's extremely fast, easy to understand, and a perfect baseline. Never underestimate the power of a simple, robust baseline.

### ✏️ Your Exercise:

Complete the `classify_regex` function below. We've given you a `keywords` dictionary to start with. Your function should take a `query`, convert it to lowercase, and check if any keywords for an intent are present. It should return the *first* intent it matches.

In [4]:
import re

# --- Dictionnaire de mots-clés amélioré ---
# → chaque entrée peut contenir :
#   - des mots simples (str)
#   - des tuples de mots (pour exiger un contexte)
#   - et des poids pour les cas ambigus
keywords = {
    "get_schedule": {
        "terms": [
            "planning", "emploi du temps", "cours", "salle", "td", "cm"
        ],
        "weight": 1.0
    },
    "check_email": {
        "terms": [
            "mail", "messagerie", "email", "boite de réception"
        ],
        "weight": 1.0
    },
    "register_classes": {
        "terms": [
            "inscription", "ip", "péda", "inscrire", ("inscription", "cours")
        ],
        "weight": 1.0
    },
    "get_student_card": {
        "terms": [
            ("carte", "étudiante"), ("carte", "amu"), "refaire ma carte",
            # "amu" seul est ambigu, donc on lui donne un poids faible
            ("carte",),  # toléré seul
            ("perdu", "carte"), ("imprimer", "carte")
        ],
        "weight": 1.2
    },
    "find_library_info": {
        "terms": [
            "bu", "bibliothèque", "livre", "emprunter", ("horaire", "bu")
        ],
        "weight": 1.0
    }
}


def classify_regex(query):
    """
    Version améliorée de la classification par regex.
    - Détection insensible à la casse
    - Correspondances de mots exactes
    - Gestion de contextes (paires de mots)
    - Pondération des scores pour gérer les ambiguïtés
    """
    query_low = query.lower()
    best_intent = "unknown"
    best_score = 0.0

    for intent, data in keywords.items():
        score = 0.0
        for term in data["terms"]:
            # --- Cas 1 : terme simple ---
            if isinstance(term, str):
                # Recherche de mot complet insensible à la casse
                if re.search(rf"\b{re.escape(term)}\b", query_low):
                    score += 1.0

            # --- Cas 2 : tuple = contexte ---
            elif isinstance(term, tuple):
                if all(re.search(rf"\b{re.escape(t)}\b", query_low) for t in term):
                    # bonus si plusieurs mots du contexte sont trouvés
                    score += len(term) * 1.0

        # pondération de l'intent
        score *= data.get("weight", 1.0)

        # on garde le meilleur score
        if score > best_score:
            best_score = score
            best_intent = intent

    # Si aucun mot-clé n'est trouvé, on retourne "unknown"
    return best_intent if best_score > 0 else "unknown"


### Exploration:
Look at the `keywords` for `get_student_card`. The keyword `"amu"` is present, but what if the query is *"Comment contacter le secrétariat d'AMU?"*? This would be a false positive! How would you make your regex more specific to avoid this? (This is the fundamental limit of regex).

---
## Module 3: Classifier 2 - Embeddings + Logistic Regression

This is our "classic ML" approach. We will use a powerful pretrained model from Hugging Face to **extract features** (embeddings) from our text. Then, we'll feed these numerical features into a very simple and fast `scikit-learn` classifier, `LogisticRegression`.

This is a perfect example of mixing the "low-level" `transformers` world with the `scikit-learn` ecosystem.

### Step 3.1: Load the Embedding Model

We'll use a `sentence-transformer` model. These models are specifically fine-tuned to create high-quality embeddings for tasks like comparison and classification. We'll use a multilingual one since our queries are in French.

### ✏️ Your Exercise:

Load the `sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2` model. We'll create a helper function `get_embedding` that takes a list of texts and returns their embeddings.

In [5]:
from sentence_transformers import SentenceTransformer
import numpy as np

# --- Chargement du modèle ---
print("Loading embedding model...")
embed_model = SentenceTransformer("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")
print("Model loaded!")

# --- Fonction utilitaire pour encoder du texte ---
def get_embeddings(text_list):
    """
    Convertit une liste de textes en embeddings vectoriels.
    """
    # encode() gère automatiquement tokenisation + passage dans le modèle + pooling
    return embed_model.encode(text_list, show_progress_bar=False, convert_to_numpy=True)

# --- Test ---
example_texts = [train_data[0]['text'], train_data[3]['text']]
example_embeddings = get_embeddings(example_texts)

print(f"\nSuccessfully converted {len(example_texts)} texts into embeddings.")
print(f"Shape of embeddings: {example_embeddings.shape}")


Loading embedding model...
Model loaded!

Successfully converted 2 texts into embeddings.
Shape of embeddings: (2, 384)


### Step 3.2: Train the Classifier

Now we'll use our `train_data` to train a `LogisticRegression` model.

### Your Exercise:
1.  Create `X_train` by getting the embeddings for all texts in `train_data`.
2.  Create `y_train` by getting the corresponding labels from `train_data`.
3.  Initialize and `fit` a `LogisticRegression` classifier.

In [6]:
from sklearn.linear_model import LogisticRegression

# --- 1. Créer X_train et y_train ---
print("Creating training embeddings...")
train_texts = [item['text'] for item in train_data]
X_train = get_embeddings(train_texts)
y_train = [item['label'] for item in train_data]

print(f"Created X_train with shape {X_train.shape} and y_train with {len(y_train)} labels")

# --- 2. Entraînement du modèle ---
print("Training Logistic Regression classifier...")
clf_logreg = LogisticRegression(max_iter=1000, solver='lbfgs', multi_class='auto')
clf_logreg.fit(X_train, y_train)
print("Classifier trained!")

# --- 3. Test rapide ---
test_query = test_data[0]['text']
test_embedding = get_embeddings([test_query])
prediction = clf_logreg.predict(test_embedding)[0]

print(f"\nQuery: '{test_query}'")
print(f"Prediction: {prediction}")
print(f"Correct: {prediction == test_data[0]['label']}")


Creating training embeddings...
Created X_train with shape (14, 384) and y_train with 14 labels
Training Logistic Regression classifier...
Classifier trained!

Query: 'C'est quand mon prochain TD?'
Prediction: get_schedule
Correct: True


/home/administrateur/.local/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1272: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.8. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


### Exploration:
How much better does this model get with more data? Try adding 5-10 more examples to `train_data` and re-run this module. Does the accuracy on the test set improve? What other `scikit-learn` classifiers could you try instead of `LogisticRegression`? (e.g., `SVC`, `RandomForestClassifier`).

---
## Module 4: Classifier 3 - The Zero-Shot Pipeline

This is the **high-level API** from your lecture. We'll use a `zero-shot-classification` pipeline. This model was trained on a Natural Language Inference (NLI) task, which allows it to determine if a "premise" (our query) entails a "hypothesis" (our candidate labels).

**The best part?** No training data required! 

### Your Exercise:
1.  Load a `zero-shot-classification` pipeline. We'll use `MoritzLaurer/mDeBERTa-v3-base-mnli-xnli`, which is a strong multilingual model.
2.  Define your `candidate_labels`. **Pro-tip**: These are *descriptions*, not just the short names. This helps the model.
3.  Call the classifier on a test query.

In [7]:
from transformers import pipeline
import torch

# --- 1. Charger le pipeline Zero-Shot ---
print("Loading zero-shot pipeline... (This may take a moment)")
clf_zero_shot = pipeline(
    "zero-shot-classification",
    model="MoritzLaurer/mDeBERTa-v3-base-mnli-xnli",
    device=0 if torch.cuda.is_available() else -1  # GPU si dispo
)
print("Pipeline loaded!")

# --- 2. Définition des labels descriptifs ---
candidate_labels = [
    "student's class schedule",
    "check student email",
    "register for new classes",
    "manage student ID card",
    "find library information"
]

# Mapping vers les labels courts du dataset
label_map_zero_shot = {
    "student's class schedule": "get_schedule",
    "check student email": "check_email",
    "register for new classes": "register_classes",
    "manage student ID card": "get_student_card",
    "find library information": "find_library_info"
}

# --- 3. Tester sur un exemple ---
test_query = test_data[0]['text']
result = clf_zero_shot(test_query, candidate_labels)

# Le label le plus probable est le premier dans result['labels']
top_label_desc = result['labels'][0]
prediction = label_map_zero_shot[top_label_desc]

print(f"\nQuery: '{test_query}'")
print(f"Model's top choice: '{top_label_desc}' (Score: {result['scores'][0]:.2f})")
print(f"Prediction: {prediction}")
print(f"Correct: {prediction == test_data[0]['label']}")


Loading zero-shot pipeline... (This may take a moment)


Device set to use cpu


Pipeline loaded!

Query: 'C'est quand mon prochain TD?'
Model's top choice: 'student's class schedule' (Score: 0.34)
Prediction: get_schedule
Correct: True


### Exploration:
This model's performance is *highly* dependent on the `candidate_labels`. What happens if you use our short labels (e.g., `"get_schedule"`)? What if you use French descriptions (e.g., `"emploi du temps"`)? Which works best?

---
## Module 5: Classifier 4 - The LLM Prompt

Time for the SOTA (State-of-the-Art) approach. We'll use a powerful LLM. Instead of *training* it, we will *prompt* it.

This is **In-Context Learning**. We'll tell the model what its job is, give it the list of possible labels, and ask it to classify our query. This is the most flexible approach, but often the slowest.

### Your Exercise:
1.  Create a prompt using our template.
2.  Call the pipeline and **parse the output** to get just the label.

### Exploration:
Prompt engineering is an art. How does the model's accuracy change if you change the system prompt? What if you give it two examples in the prompt (this is called "few-shot" prompting)?

---
## Module 6: The Showdown!

It's time to compare our four champions. We will loop through our entire `test_data` and run all four classifiers on each query. We'll record their `prediction` and their `latency` (speed).

### Your Exercise:
Fill in the loop below. We've provided the structure. You need to call each of your classifiers and time them using `time.perf_counter()`.

**Note:** For a fair *latency* comparison, we should test on a CPU. But for the *accuracy* part, using a GPU for the slow models is fine. For simplicity, we'll just test on whatever device you have. Be aware that the LLM/Zero-Shot latencies will be *much* lower on a GPU.

In [8]:
results = []
print(f"Running benchmarks on {len(test_data)} test items...")

for item in test_data:
    query = item['text']
    correct_label = item['label']

    # --- 1. Regex ---
    start_time = time.perf_counter()
    pred_regex = classify_regex(query)
    end_time = time.perf_counter()
    results.append({
        "classifier": "Regex",
        "query": query,
        "prediction": pred_regex,
        "correct": pred_regex == correct_label,
        "latency_ms": (end_time - start_time) * 1000
    })

    # --- 2. Embed + LogReg ---
    start_time = time.perf_counter()
    query_embedding = get_embeddings([query])
    pred_logreg = clf_logreg.predict(query_embedding)[0]
    end_time = time.perf_counter()
    results.append({
        "classifier": "Embed + LogReg",
        "query": query,
        "prediction": pred_logreg,
        "correct": pred_logreg == correct_label,
        "latency_ms": (end_time - start_time) * 1000
    })

    # --- 3. Zero-Shot Pipeline ---
    start_time = time.perf_counter()
    res_zero_shot = clf_zero_shot(query, candidate_labels)
    pred_zero_shot = label_map_zero_shot[res_zero_shot['labels'][0]]
    end_time = time.perf_counter()
    results.append({
        "classifier": "Zero-Shot Pipe",
        "query": query,
        "prediction": pred_zero_shot,
        "correct": pred_zero_shot == correct_label,
        "latency_ms": (end_time - start_time) * 1000
    })

print("Benchmarks complete!")

# Convert to a DataFrame for analysis
results_df = pd.DataFrame(results)

display(results_df)

Running benchmarks on 10 test items...
Benchmarks complete!


,classifier,query,prediction,correct,latency_ms
0,Regex,C'est quand mon prochain TD?,get_schedule,True,0.997256
1,Embed + LogReg,C'est quand mon prochain TD?,get_schedule,True,41.899426
2,Zero-Shot Pipe,C'est quand mon prochain TD?,get_schedule,True,42026.479859
3,Regex,Ouvrir la boite de réception,check_email,True,0.103135
4,Embed + LogReg,Ouvrir la boite de réception,check_email,True,28.836591
5,Zero-Shot Pipe,Ouvrir la boite de réception,get_student_card,False,41315.248533
6,Regex,Je veux m'inscrire en L3,register_classes,True,0.100680
7,Embed + LogReg,Je veux m'inscrire en L3,register_classes,True,27.436551
8,Zero-Shot Pipe,Je veux m'inscrire en L3,register_classes,True,41561.191368
9,Regex,Ma carte est cassée,get_student_card,True,0.223403


### Final Analysis: Accuracy and Speed

Now for the final step. Let's group by our classifiers and calculate two things:
1.  **Accuracy**: The mean of the `correct` column (True=1, False=0).
2.  **Avg. Latency**: The mean of the `latency_ms` column.

In [9]:
# We have to fix any LLM predictions that didn't give a valid label
possible_labels = train_df['labels'].unique().tolist()
valid_labels = set(possible_labels)
def validate_label(row):
    if row['classifier'] == 'LLM Prompt' and row['prediction'] not in valid_labels:
        return False # Mark as incorrect if the label isn't in our list
    return row['correct']

results_df['correct'] = results_df.apply(validate_label, axis=1)

# Now, let's calculate our final metrics
final_report = results_df.groupby('classifier').agg(
    Accuracy=pd.NamedAgg(column='correct', aggfunc='mean'),
    Avg_Latency_ms=pd.NamedAgg(column='latency_ms', aggfunc='mean')
).sort_values(by='Accuracy', ascending=False)

# Format for nice printing
final_report['Accuracy'] = final_report['Accuracy'].apply(lambda x: f"{x*100:.2f}%")
final_report['Avg_Latency_ms'] = final_report['Avg_Latency_ms'].apply(lambda x: f"{x:.2f} ms")

display(Markdown("## Final Report Card"))
display(final_report)

KeyError: 'labels'

## Conclusion & Your Turn

Look at your final report. What do you see?

* **Regex** is by far the **fastest**, but likely the least accurate (and will get worse as queries get more complex).
* **Embed + LogReg** is the perfect balance: **very fast** (once trained) and **very accurate**. s Downsides are that it needs good training data and adding new intent requires new training. 
* **Zero-Shot Pipe** is amazing for a **prototype** (good accuracy, no training!), but it's much slower.
* **LLM Prompt** is likely the **most accurate** and flexible (it can handle typos and complex phrasing!), but it is by far the **slowest**.

### Final Question:

If you were building the *real* AMU chatbot, which would you choose? 

*(Hint: There's no single right answer. A great system might use a **hybrid**! Try Regex first for simple keywords, and if it doesn't find a match, pass the query to the `Embed + LogReg` model. This gives you the speed of Regex and the power of ML!)*